---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [9]:
import pandas as pd
import numpy as np
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)


def get_all_dates():
    all_dates_first = []
    all_dates_second = []
    for t in df:
        curr = re.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]?[ ]?(?:\d{1,2}[,]? )?[12]\d{3}',t)
        curr2 = re.findall(r"\d{1,2}[/-]\d{1,2}[/-](?:\d{2}(?!\d)|[12]\d{3})|(?:\d{1,2}[/-])?[12]\d{3}", t)
        if len(curr) == 0:
            all_dates_second.append(curr2)
        else:
            all_dates_first.append(curr)

    print(len(all_dates_first))
    print(len(all_dates_second))
    print(len(df))

    return


def get_all_dates_named():
    i = 0
    all_dates = []
    for t in df:
        curr2 = re.match(r"(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2}(?!\d)|[12]\d{3})|(?:(?P<month1>\d{1,2})[/-])?(?P<year1>[12]\d{3})", t)
        if curr2 != None:
            print(str(i) + " " + t.strip())
            print(curr2.groups())
            print()
        i += 1

    return all_dates


def date_sorter():
    # t = "Lithium 0.25 (7/11/77). (7/11/77) LFTS wnl.  Urine tox neg.  Serum tox + fluoxetine 500; otherwise neg.  TSH 3.28.  BUN/Cr: 16/0.83.  Lipids unremarkable.  B12 363, Folate >20.  CBC: 4.9/36/308 Pertinent Medical Review of Systems Constitutional:"
    # curr2 = re.compile(r"(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2}(?!\d)|[12]\d{3})")
    # for i in curr2.finditer(t):
    #     print(i.groupdict())

    curr0 = re.compile(r'((?P<day>\d{1,2}) )?(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]?[ ]?((?P<day1>\d{1,2})[,]? )?(?P<year>[12]\d{3})')
    curr1 = re.compile(r"(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2}(?!\d)|[12]\d{3})")
    curr2 = re.compile(r"(?:(?P<month>\d{1,2})[/-])?(?P<year>[12]\d{3})")

    ans = []

    y = 0

    for t in df[0:500]:
        count0 = len(curr0.findall(t))
        count1 = len(curr1.findall(t))
        count2 = len(curr2.findall(t))

        if count0 != 0:
            for i in curr0.finditer(t):
                # print("curr0 ", end="")
                # print(i.groupdict())
                d = i.groupdict()
                curr = convert_date(y, year=d["year"], month=None, monthname=d["month"], day=d["day"], day1=d["day1"])
        elif count1 != 0:
            for i in curr1.finditer(t):
                # print("curr1 ", end="")
                # print(i.groupdict())
                d = i.groupdict()
                curr = convert_date(y, year=d["year"], month=d["month"], monthname=None, day=d["day"], day1=None)
        else:
            for i in curr2.finditer(t):
                # print("curr2 ", end="")
                # print(i.groupdict())
                d = i.groupdict()
                curr = convert_date(y, year=d["year"], month=d["month"], monthname=None, day=None, day1=None)
        y += 1
        ans.append(curr)
    # ans = np.array(ans)
    # ans = ans[ans[:, 3].argsort(kind="mergesort")]
    # ans = ans[ans[:, 2].argsort(kind="mergesort")]
    # ans = ans[ans[:, 1].argsort(kind="quicksort")]
    # for i in ans:
    #     print(i)


    ans = pd.DataFrame(data=ans)
    ans.columns = ["d-index", "year", "month", "day", "total"]
    # print()
    ans = ans.sort_values(["total"])
    return pd.Series(data=list(ans["d-index"]))


def get_month_number(monthname):
    name = monthname[0:3].lower()
    count = 0
    if name == "jan":
        return 1
    if name == "feb":
        return 2
    if name == "mar":
        return 3
    if name == "apr":
        return 4
    if name == "may":
        return 5
    if name == "jun":
        return 6
    if name == "jul":
        return 7
    if name == "aug":
        return 8
    if name == "sep":
        return 9
    if name == "oct":
        return 10
    if name == "nov":
        return 11
    if name == "dec":
        return 12
    return 0


def convert_date(index, year, month=None, monthname=None, day=None, day1=None):
    y = int(year)
    if y < 100:
        y += 1900
    m = 1

    if month is None and monthname is None:
        m = 1
    elif month is None:
        m = get_month_number(monthname)
    elif monthname is None:
        m = int(month)

    d = 0
    if day is None and day1 is None:
        d = 1
    elif day is None:
        d = int(day1)
    elif day1 is None:
        d = int(day)

    return [index, y, m, d, y * 365 + m * 31 + d]


date_sorter()
# print(df[72])


0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64